## Integer Encoding

- 단어 토큰화 또는 형태소 토큰화를 수행했다면 각 단어에 고유한 정수를 부여한다.

- 이때 중복은 허용하지 않는다.

- 중복이 허용되지 않는 모든 단어들의 집합을 단어 집합(Vocabulary)이라고 한다.

참고 : https://wikidocs.net/31766

In [1]:
text = "A barber is a person. a barber is good person. a barber is huge person. \
        he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. \
        His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. \
        the barber went up a huge mountain."

In [2]:
# nltk를 활용하여 문장 토큰화
from nltk.tokenize import sent_tokenize
text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [3]:
from nltk.corpus import stopwords

In [4]:
sentences = []
stop_words = set(stopwords.words('english'))

In [5]:
from nltk.tokenize import word_tokenize

In [6]:
for i in text :
    sentence = word_tokenize(i) # 단어 토큰화
    result = []
    
    for word in sentence :
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄임
        if word not in stop_words : # 불용어 제거
            if len(word) > 2 : # 단어 길이가 2이하인 경우에 추가로 단어 제거
                result.append(word)
    sentences.append(result)
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


## Build Vocabulary

In [7]:
from collections import Counter

In [8]:
words = sum(sentences, []) # 1차원 리스트로 변환해주고
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [9]:
# Counter를 사용해 단어의 모든 빈도를 계산
vocab = Counter(words)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [10]:
print(vocab['barber']) # 'barber'라는 단어의 빈도수를 출력

8


- 빈도수가 높은 순서대로 정렬하고, 빈도 수가 높을 수록 낮은 수의 정수를 부여한다.

- 이렇게 진행할 경우 빈도 수가 낮은 단어들을 제외시키면서 단어 집합의 크기를 조절하기가 편해진다.

In [11]:
# 빈도수가 높은 순서대로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x : x[1] , reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [12]:
# 이제 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여한다.
word2idx = {}
i = 0
for (word,frequency) in vocab_sorted :
    if frequency >  1 : # 빈도 수가 1일 경우 제외 
        i = i + 1 # 인코딩 번호 +1
        word2idx[word] = i
print(word2idx)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


**텍스트 데이터에 있는 단어를 모두 사용하기 보다 빈도수가 가장 높은 n개의 단어만 사용해야 할 때도 있다.**<br>
ex) 네이버 영화 리뷰 20만개중 단어 집합이 37000개일 경우


위 단어들은 빈도수가 높은 순서대로 낮은 정수가 부여되어 있으므로 빈도수 상위 n개의 단어만 사용하고 싶다면 vocab에서 정수값이 1부터 n까지인 단어들만 사용하면 된다.

In [13]:
vocab_size = 5
words_frequency = [w for w,c in word2idx.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거
print(words_frequency) 
for w in words_frequency : # 해당 단어들의 정보를 삭제
    del word2idx[w]
print(word2idx)

['word', 'keeping']
{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


## Padding

- 모든 문장에 대해서 정수 인코딩을 수행했을 때 길이는 서로 다를 수 있다.

- 이때 가상의 단어를 추가하여 길이를 맞춰준다.

- 이렇게 하면 기계가 이를 병렬 연산할 수 있다.

## One - Hot - Encoding

- **원-핫 인코딩은 전체 단어 집합의 크기(중복은 카운트하지 않은 단어들의 개수)를 벡터의 차원으로 가진다.**

- 각 단어에 고유한 정수 인덱스를 부여하고, 해당 인덱스의 원소는 1, 나머지 원소는 0을 가지는 벡터로 만든다.

## Document Term Matrix - DTM

- DTM은 마찬가지로 벡터가 단어 집합의 크기를 가지며, 대부분의 원소가 0을 가진다.

- 각 단어는 고유한 정수 인덱스를 가지며, 해당 단어가 등장 횟루를 해당 인덱스의 값으로 가진다.